In [19]:
from __future__ import print_function
import requests
import numpy as np
import random

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [55]:
def get_ans():
	'''Asks user for input. If the user types yes true is returned'''
	ans = input()
	if ans.lower == 'y' or ans.lower == 'yes' or ans == '1':
		return True
	else:
		return False

def get_wcif(comp):
	'''Pulls WCIF from WCA and return json object. If there is a problem with the request the program terminates'''
	print('Attempting to pull competition info...')
	# Send GET request for WCIF
	WCIF = requests.get('https://www.worldcubeassociation.org/api/v0/competitions/' + comp + '/wcif/public')
	# If response code is 200 everything good, otherwise something bad
	if WCIF.status_code == 200:
		print('Success!')
		return WCIF.json()
	else:
		print('Response Error!')
		exit()
        
        
# Dictionaries to translate event ID to event name
# First element: LaTeX. Second element: plain text.
event_dict = {
	'222': ['2$\\times$2$\\times$2 Cube', '2x2x2 Cube'],
	'333': ['3$\\times$3$\\times$3 Cube', '3x3x3 Cube'],
	'444': ['4$\\times$4$\\times$4 Cube', '4x4x4 Cube'],
	'555': ['5$\\times$5$\\times$5 Cube', '5x5x5 Cube'],
	'666': ['6$\\times$6$\\times$6 Cube', '6x6x6 Cube'],
	'777': ['7$\\times$7$\\times$7 Cube', '7x7x7 Cube'],
	'333bf': ['3$\\times$3$\\times$3 Blindfolded', '3x3x3 Blindfolded'],
	'333oh': ['3$\\times$3$\\times$3 One-Handed', '3x3x3 One-Handed'],
	'333fm': ['3$\\times$3$\\times$3 Fewest Moves', '3x3x3 Fewest Moves'],
	'333mbf': ['3$\\times$3$\\times$3 Multi-Blindfolded', '3x3x3 Multi-Blindfolded'],
	'333ft': ['3$\\times$33$\\times$ with Feet', '3x3x3 with Feet'],
	'minx': ['Megaminx', 'Megaminx'],
	'clock': ['Clock', 'Clock'],
	'pyram': ['Pyraminx', 'Pyraminx'],
	'skewb': ['Skewb', 'Skewb'],
	'sq1': ['Square-1', 'Square-1'],
	'444bf': ['4$\\times$4$\\times$4 Blindfolded', '4x4x4 Blindfolded'],
	'555bf': ['5$\\times$5$\\times$5 Blindfolded', '5x5x5 Blindfolded']
}

In [21]:
# Get comp name from user
print('Enter competition ID:')
#comp = input()
comp = 'Cubinginthe6ix2019'
#comp = 'NA2020'
#comp_file = comp + '-registration.csv'

WCIF = get_wcif(comp)

Enter competition ID:
Attempting to pull competition info...
Success!


In [90]:
class Person:
    def __init__(self, name, ID, events, wcaid, num_events):
        '''Person constructor'''
        self.name = name
        self.id = ID
        self.events = events
        self.wcaid = wcaid
        self.groups = [''] * num_events
        
    @staticmethod
    def build_persons(WCIF):
        '''Builds an array of competitors from WCIF data'''
        lst = []
        i = 0 # Counter
        for pers in WCIF['persons']: # Loops over each person registered
            # Build competitors
            if pers['registration'] != None:
                lst.append(Person(pers['name'], pers['registrantId'], pers['registration']['eventIds'], pers['wcaId'], len(WCIF['events'])))
                if lst[i].wcaid == None:
                    lst[i].wcaid = ''
                i += 1
        return lst
        
        
class Competition:    
    def event_group(self, eventid, index, g_size = 16):
        # Extracts persons competing in eventid
        competing = [i for i in self.competitors if eventid in i.events]
        random.shuffle(competing) # Randomized groups

        N = len(competing) # Number of competitors in event
        num_g = int(np.round(N / float(g_size))) # Number of groups

        count = 0
        for pers in competing:
            if eventid in ['333fm', '333mbf', '444bf', '555bf']: # 1 group for long events
                pers.groups[index] = '1'
                count += 1
            else:
                pers.groups[index] = str(int(count * num_g / N) + 1) # Record group
                count += 1            
            
    def group(self):
        for i in range(len(self.events)):
            self.event_group(self.events[i], i)
            
    def write_nametags(self, f_name, num_blank = 0):
        self.competitors.sort(key=lambda i: i.name) # Sort by name
        f = open(f_name, 'w')
        for pers in self.competitors:
            f.write('\\nametag{%s}{COMPETITOR}{%s}%%\n' % (pers.name, pers.wcaid))
        for i in range(num_blank): # Include blank nametags for day-of registrations
            f.write('\\nametag{}{COMPETITOR}{}%\n')
        f.close()
        
    def write_tex_groups(self, f_name):
        self.competitors.sort(key = lambda i: i.name) # Sort by name
        f = open(f_name, 'w')
        for pers in self.competitors:
            f.write('\\groups{%s}{' % (pers.name))
            for i in range(len(self.events)):
                f.write('%s & %s \\\\ ' % (event_dict[self.events[i]][0], pers.groups[i]))
            f.write('}% \n')
        f.close()
        
    def write_wca_groups(self, f_name):
        self.competitors.sort(key = lambda i: i.name) # Sort by name
        f = open(f_name, 'w')
        # Header
        f.write('| Name |')
        for i in self.events:
            f.write(' %s |' % (event_dict[i][1]))
        f.write('\n|' + ' --- |' * (1 + len(self.events)) + '\n')
        # Groups
        for pers in self.competitors:
            f.write('| %s |' % (pers.name))
            for i in range(len(self.events)):
                f.write(' %s |' % (pers.groups[i]))
            f.write('\n')
        f.close()
        
      
    @staticmethod
    def centi2min(centi):
        '''Converts a time in centiseconds to minute:second format'''
        minute = int(np.floor(centi / 6000)) # Compute number of minutes
        sec = int(np.ceil((centi - 6000 * minute) / 100)) # Compute remainder
        return str(minute) + ':' + str(sec).zfill(2) # Format string (zfill zero pads)
    
    @staticmethod
    def build_events(WCIF):
        '''Build a list of events held at the competition'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            lst[i] = WCIF['events'][i]['id']
        return lst
    
    @staticmethod
    def build_cutoffs(WCIF):
        '''Build a list of cutoffs for the competition. Note: Assumes cutoffs only in the first round'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['cutoff'] == None:
                lst[i] = ''
            else:
                lst[i] = Competition.centi2min(WCIF['events'][i]['rounds'][0]['cutoff']['attemptResult'])
        return lst
    
    @staticmethod
    def build_limits(WCIF):
        '''Build a list of time limits for the competition'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['cutoff'] == None:
                lst[i] = ''
            else:
                lst[i] = Competition.centi2min(WCIF['events'][i]['rounds'][0]['timeLimit']['centiseconds'])
        return lst
    
    @staticmethod
    def build_rounds(WCIF):
        '''Build a list of additional rounds for the competition'''
        lst = [[] for i in range(len(WCIF['events']))]
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['advancementCondition'] != None:
                for j in range(len(WCIF['events'][i]['rounds']) - 1):
                    lst[i].append(WCIF['events'][i]['rounds'][j]['advancementCondition']['level'])
        return lst
    
    
    def __init__(self, WCIF):
        '''Competition constructor'''
        self.name = WCIF['name']
        self.events = Competition.build_events(WCIF)
        self.cutoffs = Competition.build_cutoffs(WCIF)
        self.limits = Competition.build_limits(WCIF)
        self.rounds = Competition.build_rounds(WCIF)
        self.competitors = Person.build_persons(WCIF)
            

In [88]:
comp = Competition(WCIF)

In [89]:
comp.write_nametags('Test.tex')
comp.group()
comp.write_tex_groups('TestGroup.tex')
comp.write_wca_groups('Test.md')

In [26]:
comp.group()
print(comp.events)
for i in comp.competitors:
    print(i.name, i.groups)

['333', '444', '666', '777', '333bf']
Chen-Pin Sun (孫振斌) ['1', '4', '1', '1', '1']
Abdullah Gulab ['5', '4', '2', '2', '2']
Jonathan Esparaz ['3', '3', '2', '1', '2']
Neel Shah ['6', '5', '', '', '']
Alyssa Esparaz ['5', '3', '', '', '2']
Brady Metherall ['3', '3', '1', '1', '1']
Sarah Strong ['2', '2', '', '1', '']
Kenneth Yeung ['2', '4', '', '1', '']
Easton Pan ['6', '', '', '', '']
Shanglin Ye ['2', '2', '', '', '']
Sebastian Desaulniers ['7', '2', '1', '', '1']
Emma Kennedy ['3', '3', '', '', '']
Kyria Sztainbok ['3', '1', '', '', '2']
Zhivko Radkov Dimitrov ['2', '5', '', '', '']
Joon Song ['4', '5', '', '', '2']
Alireza Azimi (علیرضا عظیمی) ['6', '4', '', '', '']
Emmanuel Intac ['4', '', '', '', '']
Ray Vince Ong ['5', '3', '', '', '']
William Hu ['3', '1', '', '', '2']
Tin Huang ['6', '4', '', '', '1']
Kairui Huang ['1', '4', '', '', '']
Sami Mokatren ['2', '2', '', '', '']
LianCheng Guo ['6', '1', '2', '2', '2']
Can Çetin ['2', '1', '1', '', '1']
Tarandeep Mittal ['2', '5', ''